# Auto post to Social site
Schedule: 1 Image post per day per Page (setup n cron items, each item has 1 category in param)

In [41]:
from PIL import Image, ImageDraw, ImageFont
import pymongo
import os
import requests
import time

In [42]:
from dotenv import load_dotenv
load_dotenv(override=True) 
LI_URI = os.environ['LI_URI']
LI_REST_URI = os.environ['LI_REST_URI']
LI_VERSION = os.environ['LI_VERSION']

In [43]:
def get_timestamp():
    return str(int(time.time()))

In [44]:
#current path of this project
CURRENT_PATH = '/Users/sang/Documents/Source/Python/python_webscrap/cert_exam/'

In [45]:
def wrap_text(text, font, max_width, draw):
    """
    Splits text into lines that fit within max_width
    """
    words = text.split()
    lines = []
    current_line = ""

    for word in words:
        test_line = current_line + (" " if current_line else "") + word
        w = draw.textlength(test_line, font=font)

        if w <= max_width:
            current_line = test_line
        else:
            lines.append(current_line)
            current_line = word

    if current_line:
        lines.append(current_line)

    return lines

In [46]:
def draw_wrapped_text(draw, position, text, font, max_width, fill=(255,255,255), line_spacing=6):
    x, y = position
    lines = wrap_text(text, font, max_width, draw)

    for line in lines:
        draw.text((x, y), line, font=font, fill=fill)
        y += font.size + line_spacing


In [47]:
#2 or 3 certifications in 1 image
LOGO_POSITIONS = [(70, 290), (70, 630), (70, 985)]
NAME_POSITIONS = [(410, 350), (410, 695), (410, 1035)]  #certs name
SALARY_POSITION = (200, 130)    #salary range
FOOTER_URL_POSITION = (330, 1305)   #https://udemy/...

IMG_TEMPLATE_PATH = CURRENT_PATH + 'template/img_template.png'
IMG_LOGO_PATH_PREFIX = CURRENT_PATH + 'logo/'

IMG_OUTPUT_PATH_PREFIX = CURRENT_PATH + 'output/'  #the final image
FONT_PATH = CURRENT_PATH + "font/Swansea-q3pd.ttf"

In [48]:
CTA_LINKS = '\n Check out practice questions for exams: \n'

In [49]:
db_client = pymongo.MongoClient(os.environ['DB_URI'])
db = db_client['db_certificates']   
tb_advertising_posts = db['tb_advertising_posts']   #store contents to post in social media
tb_cert_metadata = db['tb_cert_metadata']    #meta data of certificates
tb_linkedin_app = db['tb_linkedin_app']    #LI App
tb_linkedin_page = db['tb_linkedin_page']    #LI Page

In [50]:
def find_1_content_from_db(_category):
    return tb_advertising_posts.find_one({'posted_li_img': 0, 'category': _category})  #not yet post any image

In [51]:
def create_certifications_image(
    page_link:str,
    cert_symbols: list,
    cert_names: list,
    salary_range: str,
    output_path: str
):
    base = Image.open(IMG_TEMPLATE_PATH).convert("RGBA")
    draw = ImageDraw.Draw(base)
    index = 0
    # ---- Add logo images ----
    for symbol in cert_symbols:
        overlay_img = Image.open(IMG_LOGO_PATH_PREFIX + symbol + '.png').convert("RGBA")
        #don't need to resize because logos are saved with size 330x330
        # if "size" in item and item["size"]:
        #     overlay_img = overlay_img.resize(item["size"], Image.ANTIALIAS)
        base.paste(overlay_img, LOGO_POSITIONS[index], overlay_img)
        index = index + 1

    # ---- Add certification names ----
    index = 0
    for t in cert_names:
        font = ImageFont.truetype(FONT_PATH, 48)
        draw_wrapped_text(
            draw=draw,
            position= NAME_POSITIONS[index],
            text= t,
            font= font,
            max_width= 620,
            fill= (0, 0, 153),   #certification name color
            line_spacing=8
        )
        index = index + 1
    #draw salary range
    draw_wrapped_text(
            draw=draw,
            position= SALARY_POSITION,
            text= 'Salary range: ' + salary_range,
            font= ImageFont.truetype(FONT_PATH, 60),
            max_width= 800,
            fill= (153, 153, 0)   #text color
        )
    #draw footer
    draw_wrapped_text(
            draw=draw,
            position= FOOTER_URL_POSITION,
            text= page_link,
            font= ImageFont.truetype(FONT_PATH, 20),
            max_width= 620,
            fill= (0, 0, 255)
        )

    base.convert("RGB").save(output_path, "PNG")
    print(f"Final image saved to: {output_path}")


In [52]:
#share new post with new uploaded image
def share_my_img_2_LI(asset, page_id, access_token, post_content):
    #print('asset: ' + asset)
    payload = {
        "author": page_id,
        "lifecycleState": "PUBLISHED",
        "specificContent": {
            "com.linkedin.ugc.ShareContent": {
                "shareCommentary": {
                    "text": post_content
                },
                "shareMediaCategory": "IMAGE",
                "media": [
                    {
                        "status": "READY",
                        "media": asset
                    }
                ]
            }
        },
        "visibility": {
            "com.linkedin.ugc.MemberNetworkVisibility": "PUBLIC"
        }
    }
    li_headers = {
        'Authorization': access_token,
        'LinkedIn-Version': LI_VERSION,
        'Content-Type': 'application/json'
    }
    try:
        response = requests.post(LI_URI + 'v2/ugcPosts', json=payload, headers=li_headers)

        if response.status_code >= 200 and response.status_code < 300:
            print("The image was shared successfully!")
            return 'ok'
        else:
            print("The image was shared failed.")
            return 'failed'
    except Exception as e:
        print(e)   
        return 'failed'

In [53]:
#upload image to LI
def upload_img_2_LI(li_access_token, file_path, upload_detail, page_id, post_content):
    uploadUrl = upload_detail['uploadMechanism']['com.linkedin.digitalmedia.uploading.MediaUploadHttpRequest']['uploadUrl']
    #upload the image https://learn.microsoft.com/en-us/linkedin/consumer/integrations/self-serve/share-on-linkedin#upload-image-or-video-binary-file
    try:
        with open(file_path, 'rb') as file:
            files = {'file': (file.name, file, 'image/jpg')}
            headers = {
                'Authorization': li_access_token,
                'LinkedIn-Version': LI_VERSION
            }

            response = requests.post(uploadUrl, files=files, headers=headers)
            print("Status Code:", response.status_code)
            print("Headers:", response.headers)
            print("Response Body:", response.text)

            if response.status_code >= 200 and response.status_code < 300:
                print("File uploaded successfully!")
                time.sleep(5)   #delay 5 seconds for image going through LI system
                result = share_my_img_2_LI(upload_detail['asset'], page_id, li_access_token, post_content)
                return result
            else:
                print("File upload failed.")
                return 'failed'

    except FileNotFoundError:
        print(f"Error: File not found at path: {file_path}")
        return 'failed' 
    except requests.exceptions.RequestException as e:
        print(f"Error during upload: {e}")
        return 'failed'
    

In [54]:
#get upload link from LI
def get_LI_upload_link(page_id, li_headers):
    url = LI_URI + 'v2/assets?action=registerUpload'
    payload = {
        "registerUploadRequest": {
            "recipes": [
                "urn:li:digitalmediaRecipe:feedshare-image"
            ],
            "owner": page_id,
            "serviceRelationships": [
                {
                    "relationshipType": "OWNER",
                    "identifier": "urn:li:userGeneratedContent"
                }
            ]
        }
    }
    
    try:
        detail = requests.post(url, json=payload, headers=li_headers)
        return detail.json()
    except Exception as e:
        print(e)
        return {'error': e}

In [ ]:
def post_1_image_by_category(_category):
    post_details = find_1_content_from_db(_category)
    if post_details is None:
        print('There is no post for Image content of category: ' + _category)
        return
    if 'symbols' not in post_details:
        print('There is no symbol for Image content of category: ' + _category)
        return
    symbols = post_details['symbols'].split(',')
    #find name of certifications
    cert_names = []
    cert_links = []
    for symbol in symbols:
        cert_details = tb_cert_metadata.find_one({'symbol':symbol})
        if cert_details is None:
            print('There is no detail for the certification: ' + symbol)
            return
        cert_names.append(cert_details['name'])
        cert_links.append(cert_details['udemy_link'])
     #find app token and page linking to this category
    page_info = tb_linkedin_page.find_one({'category': _category})
    if page_info is None:
        print('There is no page info for category: ' + _category)
        return
    #create final image
    new_img_path = IMG_OUTPUT_PATH_PREFIX + post_details['symbols'].replace(',', '-') + '.png'
    create_certifications_image(page_info['link'], symbols, cert_names, post_details['salary'], new_img_path)
    #find app info
    app_info = tb_linkedin_app.find_one({'app_id': page_info['app_id']})
    if app_info is None:
        print('There is no app info for category: ' + _category)
        return
    #create CTA link
    cta_links = []
    index = 1
    for cert_name in cert_names:
        cta_links.append(str(index) + '. ' + cert_name + ': ' + cert_links[index-1] + '\n')
        index = index + 1
    cta_links_text = CTA_LINKS + ''.join(cta_links)
    post_content = post_details['content'] + cta_links_text
    #upload image to to Linkedin Page
    li_headers = {
        'Authorization': 'Bearer ' + app_info['access_token'],
        'LinkedIn-Version': LI_VERSION,
        'Content-Type': 'application/json'
    }
    upload_link = get_LI_upload_link(page_info['page_id'], li_headers)
    print('6666', upload_link)
    result_upload = upload_img_2_LI(app_info['access_token'], new_img_path, upload_link['value'], page_info['page_id'], post_content)
    if result_upload == 'ok':
        #the post is successfully up
        #delete that image
        if os.path.exists(new_img_path):
            os.remove(new_img_path)
        print('==== The image is shared in LI Page for category: ' + _category)
        #add flag to that post to indicate done
        post_details['posted_li_img'] = 1
        tb_advertising_posts.save(post_details)
        return
    else:
        print('There is an error for posting in LI page for category: ' + _category)
        return
    


In [60]:
post_1_image_by_category('CLOUD')

Final image saved to: /Users/sang/Documents/Source/Python/python_webscrap/cert_exam/output/COMP_CLOUD_PLUS-COMP_CLOUD_NETX.png
{'_id': ObjectId('69744b6d477a6b59ad741723'), 'app_id': '86sl10u3zkrucb', 'app_secret': 'WPL_AP1.Xc4vRqjtc4Z23rES.Auwc0w==', 'access_token': 'AQVNIYWacWobOC087JlhPKtTzAAA0RjCrwc55p_nfX3oCt6bPQic8dDWDkpu3yiIrsSqjAzm6Bw9pijkkuu0J0OjnqNt_8Z3PxhBcLBhjeNP8cskvwAXy2lrpM3pvAhEvv5BPIfYL0MraXf9AS18WqcZBKWyTTJEWdVhtz7RcHGcj9Dptl5KxZLkdAobaDyb-bnAffpISRcvTRQ2jPIrQt65PGn7RjTtqMdbOiRFNhxpfvYNfRF42UyKHpG8DD76PfgE1ENL-3aAcfAFV1jl_Ssns4Du8EDU9USgaQASbHU8W7MABJ_xFjAtqyU9lz9vpjcllzRLpWfebqmqoWgkT8QWG_eq5Q', 'expires_in': 5183999, 'refresh_token': 'AQUQEiUlJFPOxWMXoqZKwLx6tMvqOry10BPJzdWjP_h1uxcOXkbgKXJxrbspAA90kfwJEoqQhbwTKMzypnIhbTey4ibuLBs0Er2rroxPQyahAeJ_9TH1V5nW208kuHhOFmxGeFwoSANlHpjxEr_Zer0-exgAiKOL9OU8rcHkYa4SQ_aAOfkRpy2WJBENDXLqdaoMq1ExKYROKgBQvcIpO7mCWNVvkpxSxne5G2iJbbM-7SQFP-j-NGe6zEe501bvjeFmgZmBYfDlXGWkS7EO0W-XeIsbHB8Zk4C9ALFamW7xqUmKQDdWBnhf-DFq4m4-AsqNT2iDzmxZ2LsMO

KeyError: 'value'

In [ ]:
# if __name__ == '__main__':
#     args = sys.argv
#     print(args)
#     if len(args) > 1:
#         category = args[1]   #category of certifications
#         print(category)
#         post_1_image_by_category(category)

['/Users/sang/.pyenv/versions/3.9.10/lib/python3.9/site-packages/ipykernel_launcher.py', '--f=/Users/sang/Library/Jupyter/runtime/kernel-v3796ce355cc1e9ff6283ac92de1a2166e8015b7e8.json']
--f=/Users/sang/Library/Jupyter/runtime/kernel-v3796ce355cc1e9ff6283ac92de1a2166e8015b7e8.json
There is no post for Image content of category: --f=/Users/sang/Library/Jupyter/runtime/kernel-v3796ce355cc1e9ff6283ac92de1a2166e8015b7e8.json


In [ ]:
db_client.close()